In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [30]:
%%time

Countries = ['Portugal','Spain','England','Italy','Germany','France']


URL = 'https://www.resultados-futbol.com/ligue_1'
r = requests.get(URL)
page = r.content
soup = BeautifulSoup(page, 'html5lib')

current_year = int(soup.find('div', class_ = "titular-data").text.strip()[0:4])+1

df_predictions = pd.DataFrame()

for Country in Countries:

    if Country == 'Portugal':
        country = 'portugal'
    elif Country == 'Spain':
        country = 'primera'
    elif Country == 'England':
        country = 'premier'
    elif Country == 'Italy':
        country = 'serie_a'
    elif Country == 'Germany':
        country = 'bundesliga'
    elif Country == 'France':
        country = 'ligue_1'
    else:
        print('Country not available, please choose between Portugal,\
    Spain, England, Italy, Germany or France')


    # Last/next weekday of current year

    URL = f'https://www.resultados-futbol.com/{country}'
    r = requests.get(URL)
    page = r.content
    soup = BeautifulSoup(page, 'html5lib')
    last_weekday = soup.find_all('div', class_ = "j_cur")[1].find('a').text
    last_weekday_number = int(re.findall(r'\d+', last_weekday)[0])
    next_weekday_number = last_weekday_number+1
    
    
    # Generate the table with the matches of the current weekday
    
    URL = f'https://www.resultados-futbol.com/{country}/grupo1/jornada{next_weekday_number}'
    df_country_next_weekday = pd.read_html(URL)[7]

    
    # Edit the table

    df_country_next_weekday.columns = list(range(len(
        list(df_country_next_weekday.columns))))
    
    df_country_next_weekday = df_country_next_weekday[[1,2,4]]

    df_country_next_weekday = df_country_next_weekday[df_country_next_weekday[1]
                                                      .str.contains("Sin comenzar")]
    
    df_country_next_weekday = df_country_next_weekday.rename(
    columns = {1:'Date',2:'Home_team',4:'Away_team'})
    
    df_country_next_weekday['Date'] = df_country_next_weekday['Date'].apply(
    lambda x: x[0:9])
    
    df_country_next_weekday.insert(0,'Year',current_year)
    
    df_country_next_weekday.insert(1,'Country',Country)
    
    df_country_next_weekday['Weekday'] = next_weekday_number
    
    
    df_predictions = pd.concat([df_predictions,df_country_next_weekday],
                              ignore_index = True, axis = 0)

df_predictions.head(3)

CPU times: user 13.6 s, sys: 70.1 ms, total: 13.6 s
Wall time: 22.3 s


,Year,Country,Date,Home_team,Away_team,Weekday
0,2022,Portugal,10 Dic 21,Paços de Ferreira,Gil Vicente,14
1,2022,Portugal,11 Dic 21,Marítimo,CD Santa Clara,14
2,2022,Portugal,11 Dic 21,Vitória Guimarães,Tondela,14


In [31]:
df = pd.read_excel('Merged_table_from_2005.xlsx')
df.drop(['Unnamed: 0'],axis = 1, inplace = True)
df

,Year,Country,Date,Competition,Home_team,Away_team,Result,1x2,number_of_games_last_days_home_team,number_of_games_last_days_away_team,...,Home_team_OVR,Home_team_Rival_team,Home_team_Budget_Mill_€,Away_team_ATT,Away_team_MID,Away_team_DEF,Away_team_OVR,Away_team_Rival_team,Away_team_Budget_Mill_€,Rivals
0,2005,Portugal,22 May 05,National League,Boavista,Benfica,1 - 1,x,2,2,...,58,Vitória SC,0.375,80,74,74,75,Sporting Lisbon,5.0,0
1,2005,Portugal,07 May 05,National League,Penafiel,Benfica,1 - 0,1,3,4,...,61,FC Porto,0.100,80,74,74,75,Sporting Lisbon,5.0,0
2,2005,Portugal,24 Apr 05,National League,Estoril,Benfica,1 - 2,2,3,4,...,59,Vitória Futebol Clube,2.000,80,74,74,75,Sporting Lisbon,5.0,0
3,2005,Portugal,10 Apr 05,National League,Rio Ave,Benfica,1 - 0,1,2,1,...,52,Benfica,2.900,80,74,74,75,Sporting Lisbon,5.0,1
4,2005,Portugal,19 Mar 05,National League,Vitória Setúbal,Benfica,0 - 2,2,4,4,...,61,Belenenses,0.350,80,74,74,75,Sporting Lisbon,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36457,2022,France,29 Aug 21,National League,Troyes,Monaco,1 - 2,2,2,5,...,72,Stade de Reims,6.000,81,77,77,78,OGC Nice,26.0,0
36458,2022,France,13 Aug 21,National League,Lorient,Monaco,1 - 0,1,1,3,...,72,Stade Rennais,7.000,81,77,77,78,OGC Nice,26.0,0
36459,2022,France,01 Dec 21,National League,Angers SCO,Monaco,1 - 3,2,2,3,...,74,En Avant Guingamp,6.500,81,77,77,78,OGC Nice,26.0,0
36460,2022,France,26 Sep 21,National League,Clermont,Monaco,1 - 3,2,3,4,...,71,AS Saint-Étienne,4.500,81,77,77,78,OGC Nice,26.0,0


In [ ]:
%%time

# Calculate the number of games played by each team in the last days.

import datetime
from datetime import datetime
from datetime import timedelta

last_days = 21

datetime_date_list = []

for date in table_all_years['Date']:
    date_object = datetime.strptime(date, "%d %b %y")
    datetime_date_list.append(date_object)

table_all_years.insert(list(table_all_years.columns).index('Date')+1,'Datetime_date_list', datetime_date_list)



number_of_games_last_days_home_team_list = []

for i in range(table_all_years.shape[0]):
    Team = table_all_years['Home_team'][i]
    match_day = table_all_years['Datetime_date_list'][i]
    match_day_minus_last_days = match_day - timedelta(days = last_days)

    number_of_games_last_days_home_team_list.append(
        sum(
    (table_all_years.Datetime_date_list >= match_day_minus_last_days) & 
    (table_all_years.Datetime_date_list < match_day) & 
    ((table_all_years.Home_team == Team) | (table_all_years.Away_team == Team)) & 
    (table_all_years.Competition != 'Friendly match')
    )
    )

table_all_years['number_of_games_last_days_home_team'] = number_of_games_last_days_home_team_list


number_of_games_last_days_away_team_list = []

for i in range(table_all_years.shape[0]):
    Team = table_all_years['Away_team'][i]
    match_day = table_all_years['Datetime_date_list'][i]
    match_day_minus_last_days = match_day - timedelta(days = last_days)

    number_of_games_last_days_away_team_list.append(
        sum(
    (table_all_years.Datetime_date_list >= match_day_minus_last_days) & 
    (table_all_years.Datetime_date_list < match_day) & 
    ((table_all_years.Away_team == Team) | (table_all_years.Home_team == Team)) & 
    (table_all_years.Competition != 'Friendly match')
    )
    )

table_all_years['number_of_games_last_days_away_team'] = number_of_games_last_days_away_team_list

# Note: teams outside the 6 national leagues will appear in international
# games with 0 or very little games in the last days. However it should not
# be a problem, as we are not predicting game results of their national leagues.

table_all_years